In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'IPL_Ball_by_Ball_2008_2022.csv')

In [3]:
df.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [4]:
df.isnull().sum()

ID                        0
innings                   0
overs                     0
ballnumber                0
batter                    0
bowler                    0
non-striker               0
extra_type           213905
batsman_run               0
extras_run                0
total_run                 0
non_boundary              0
isWicketDelivery          0
player_out           214803
kind                 214803
fielders_involved    217966
BattingTeam               0
dtype: int64

In [5]:
powerplay_df = df[df['overs'].between(1, 6)]

In [6]:
powerplay_runs = powerplay_df.groupby(['ID', 'BattingTeam'])['total_run'].sum().reset_index(name='powerplay_runs')

In [7]:
powerplay_balls = powerplay_df[(powerplay_df['extra_type'] != 'wides') & (powerplay_df['extra_type'] != 'noballs')].groupby(['ID', 'BattingTeam']).size().reset_index(name='legal_balls')

In [8]:
powerplay_data = pd.merge(powerplay_runs, powerplay_balls, on=['ID', 'BattingTeam'])

In [9]:
powerplay_data['run_rate'] = powerplay_data['powerplay_runs'] / (powerplay_data['legal_balls'] / 6)

In [10]:
powerplay_data

,ID,BattingTeam,powerplay_runs,legal_balls,run_rate
0,335982,Kolkata Knight Riders,65,36,10.833333
1,335982,Royal Challengers Bangalore,29,36,4.833333
2,335983,Chennai Super Kings,57,36,9.500000
3,335983,Kings XI Punjab,60,36,10.000000
4,335984,Delhi Daredevils,51,36,8.500000
...,...,...,...,...,...
1893,1312198,Royal Challengers Bangalore,56,36,9.333333
1894,1312199,Rajasthan Royals,61,36,10.166667
1895,1312199,Royal Challengers Bangalore,44,36,7.333333
1896,1312200,Gujarat Titans,30,36,5.000000


In [11]:
from scipy.stats import chi2_contingency


In [13]:
matches_df = pd.read_csv(r'IPL_Matches_2008_2022.csv')

In [14]:
powerplay_data = pd.merge(powerplay_data, matches_df[['ID', 'WinningTeam']], on='ID')

In [15]:
powerplay_data['run_rate_category'] = pd.cut(powerplay_data['run_rate'], bins=[0, 6, 8, 12], labels=['Low', 'Medium', 'High'])

In [16]:
powerplay_data['won_match'] = powerplay_data['BattingTeam'] == powerplay_data['WinningTeam']

In [17]:
contingency_table = pd.crosstab(powerplay_data['run_rate_category'], powerplay_data['won_match'])

In [18]:
chi2, p, dof, ex = chi2_contingency(contingency_table)

In [19]:
dof

2

In [20]:
ex

array([[193.07933081, 185.92066919],
       [361.19589854, 347.80410146],
       [389.72477064, 375.27522936]])

In [27]:
print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")

Chi-square statistic: 71.34874129963089
P-value: 3.2123117140437523e-16
